In [ ]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *

In [ ]:
directory_targ = directory_project
directory_targ = directory_targ + "fig1/"

update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
stds = np.load("rmses.npy") / 2

intervals = np.load("intervals.npy")


def find_index(r):
    if r <= intervals[0]:
        return 0
    for i in range(len(intervals) - 1):
        if r >= intervals[i] and r < intervals[i + 1]:
            return i
    return len(intervals) - 2  # Returns -1 if no such index exists


def noise_fun(r):
    index = find_index(r)
    std = stds[index]
    # return(np.random.normal(0,0.7))
    return np.random.normal(0, std)

In [ ]:
plates = all_folders["unique_id"].unique()
errors = []
for plate in plates:
    folders = all_folders.loc[all_folders["unique_id"] == plate]
    folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]
    folders = folders.sort_values(by="datetime")

    exp = Experiment(directory_targ)
    i = 0
    exp.load(folders.iloc[i : i + 1], suffix="_width")
    for t in range(exp.ts):
        exp.load_tile_information(t)
    G = exp.nx_graph[t]
    components = nx.connected_components(G)

    # Find the largest connected component
    largest_component = max(components, key=len)

    # Create a new graph representing the largest connected component
    largest_component_graph = G.subgraph(largest_component)
    exp.nx_graph[t] = largest_component_graph
    edges = get_all_edges(exp, t)
    lengths = [edge.length_um(t) for edge in edges]
    radiuses = [edge.width(t) / 2 for edge in edges]
    radiuses = np.array(radiuses)
    noises = np.array([noise_fun(r) for r in radiuses])
    biovolume1 = np.sum(radiuses**2 * np.pi * lengths)
    biovolume2 = np.sum((radiuses + noises) ** 2 * np.pi * lengths)
    print(biovolume1)
    errors.append((biovolume2 - biovolume1) / biovolume1)
    # break

In [ ]:
fig, ax = plt.subplots()
ax.hist(np.array(errors) * 100)

In [ ]:
np.mean(errors)

In [ ]:
vmax = 12
vmin = 2
plot_edge_color_value(
    exp,
    t,
    lambda edge: edge.width(t),
    cmap=cm.get_cmap("cool", 100),
    v_min=vmin,
    v_max=vmax,
    plot_cmap=True,
    show_background=False,
    dilation=10,
    figsize=(12, 7),
)

In [ ]:
stds

In [ ]:
radiuses = np.array(radiuses)
noises = np.array([noise_fun(r) for r in radiuses])

In [ ]:
noises

In [ ]:
biovolume1 = np.sum(radiuses**2 * np.pi * lengths)
biovolume1

In [ ]:
biovolume2 = np.sum((radiuses + noises) ** 2 * np.pi * lengths)
biovolume2

In [ ]:
(biovolume2 - biovolume1) / biovolume1

In [ ]:
fig, ax = plt.subplots()
ax.hist(radiuses, 20, alpha=0.5)
ax.hist(radiuses + noises, 20, alpha=0.5)